In [ ]:
'''
  code by Tae Hwan Jung(Jeff Jung) @graykode, modify by wmathor
'''
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

dtype = torch.FloatTensor

In [ ]:
sentences = ["i like dog", "i love coffee", "i hate milk"]

word_list = " ".join(sentences).split()
vocab = list(set(word_list))
word2idx = {w: i for i, w in enumerate(vocab)}
idx2word = {i: w for i, w in enumerate(vocab)}
n_class = len(vocab)

# TextRNN Parameter
batch_size = 2
n_step = 2 # number of cells(= number of Step)
n_hidden = 5 # number of hidden units in one cell

def make_data(sentences):
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()
        input = [word2idx[n] for n in word[:-1]]
        target = word2idx[word[-1]]

        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)

    return input_batch, target_batch

input_batch, target_batch = make_data(sentences)
input_batch, target_batch = torch.Tensor(input_batch), torch.LongTensor(target_batch)
dataset = Data.TensorDataset(input_batch, target_batch)
loader = Data.DataLoader(dataset, batch_size, True)

In [ ]:
# print(input_batch)
# print(input_batch.shape)
# print(np.eye(7)[[1, 2, 5]])

torch.Size([3, 2, 7])


In [ ]:
class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN, self).__init__()
        self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden)
        # input_size 指的是每个单词用多少维的向量去编码
        # hidden_size 指的是输出维度是多少
        # fully connected layer
        self.fc = nn.Linear(n_hidden, n_class)

    def forward(self, hidden, X): #这里不一样！！！hidden,x
        # X: [batch_size, n_step, n_class]
        X = X.transpose(0, 1) # X : [n_step, batch_size, n_class]是pytorch规定的Rnn的输入内容
        out, hidden = self.rnn(X, hidden)
        # out : [n_step, batch_size, num_directions(=1) * n_hidden]
        # hidden : [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        out = out[-1] # [batch_size, num_directions(=1) * n_hidden] ⭐ 只取最后一个
        model = self.fc(out)
        return model

model = TextRNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training
for epoch in range(5000):
    for x, y in loader:
      # hidden : [num_layers * num_directions, batch, hidden_size]
      hidden = torch.zeros(1, x.shape[0], n_hidden) # h0
      # x : [batch_size, n_step, n_class]
      pred = model(hidden, x) #所以rnn(h0,x)

      # pred : [batch_size, n_class], y : [batch_size] (LongTensor, not one-hot)
      loss = criterion(pred, y)
      if (epoch + 1) % 1000 == 0:
          print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

Epoch: 1000 cost = 0.045840
Epoch: 1000 cost = 0.059695
Epoch: 2000 cost = 0.009017
Epoch: 2000 cost = 0.009856
Epoch: 3000 cost = 0.003041
Epoch: 3000 cost = 0.002324
Epoch: 4000 cost = 0.001052
Epoch: 4000 cost = 0.000802
Epoch: 5000 cost = 0.000340
Epoch: 5000 cost = 0.000372


In [ ]:
input = [sen.split()[:2] for sen in sentences] #取每句前两个 如果[:-1]取每句从头到倒数第一个不含倒数
# Predict
hidden = torch.zeros(1, len(input), n_hidden)
predict = model(hidden, input_batch).data.max(1, keepdim=True)[1] #取dim=1这一维度上就是最里层行的最大值，最外层就是dim=0就是列
print([sen.split()[:2] for sen in sentences], '->', [idx2word[n.item()] for n in predict.squeeze()])

[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']
